
# Нейро‑консультант автосалона «АвтоЛидер» — Colab прототип

Прототип демонстрирует **нейро‑сотрудника** на базе GPT + LangGraph + FAISS и RAG:
- офлайн‑подготовка векторной БД из Excel каталога;
- поток обработки обращений (интенты, маршрутизация);
- подбор **3 моделей** под запрос клиента из ассортимента **Chery / Jetour / Haval City**;
- **Генерация рекомендаций через LLM** исключительно по данным каталога;
- анти‑галлюцинации: LLM использует только факты из каталога;
- SLA: ежедневно 09:00–21:00 (MSK), после 20:40 — особая формулировка.



In [ ]:

# очистка конфликтов
!pip -q uninstall -y langchain langgraph-prebuilt langchain-text-splitters || true

# корректные версии под Colab (pandas 2.2.2; pydantic < 2.12; langgraph 0.2.56)
!pip -q install -U \
  pandas==2.2.2 \
  "pydantic<2.12" \
  faiss-cpu sentence-transformers openpyxl pytz \
  openai tiktoken \
  "langchain-core==0.3.75" \
  "langgraph==0.2.56" \
  reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 9.5 MB/s eta 0:00:00


## Конфигурация и ключи

In [ ]:

# === Конфигурация и ключи ===
import os, json, pytz
from datetime import datetime, time as dtime

# Попытка получить ключ из Colab Secrets
try:
    from google.colab import userdata
    secret_key = userdata.get('OPENAI_API_KEY')
    if secret_key:
        os.environ['OPENAI_API_KEY'] = secret_key
        print("✅ OPENAI_API_KEY взят из Colab Secrets.")
    else:
        print("⚠️ В Colab Secrets нет ключа OPENAI_API_KEY.")
except Exception as e:
    print("⚠️ Не удалось прочитать Colab Secrets:", e)

# Проверка
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
EMBED_MODEL = os.getenv("EMBED_MODEL", "BAAI/bge-m3")

print("OPENAI_API_KEY установлен:", bool(OPENAI_API_KEY))
print("Модель:", OPENAI_MODEL)
print("Эмбеддинги:", EMBED_MODEL)

# Проверка доступности API
try:
    from openai import OpenAI
    client = OpenAI()
    _ = client.models.list()
    print("✅ OpenAI API доступен.")
except Exception as e:
    print("❌ Ошибка доступа к OpenAI API:", e)

# Временная зона и рабочее время
TZ = pytz.timezone("Asia/Almaty")
WORK_START = dtime(9, 0, 0)
WORK_END = dtime(21, 0, 0)
LATE_CUTOFF = dtime(20, 50, 0)



✅ OPENAI_API_KEY взят из Colab Secrets.
OPENAI_API_KEY установлен: True
Модель: gpt-4o-mini
Эмбеддинги: BAAI/bge-m3
✅ OpenAI API доступен.


## Загрузка Excel‑каталога (Chery / Jetour / Haval City)

In [ ]:

from google.colab import files

print("Загрузите файл: auto_catalog_ru_FINAL_ALL_2025.xlsx")
uploaded = files.upload()  # диалог загрузки
XLSX_PATH = next(iter(uploaded.keys()))
print("Файл получен:", XLSX_PATH)


Загрузите файл: auto_catalog_ru_FINAL_ALL_2025.xlsx


Saving auto_catalog_ru_FINAL_ALL_2025.xlsx to auto_catalog_ru_FINAL_ALL_2025.xlsx
Файл получен: auto_catalog_ru_FINAL_ALL_2025.xlsx


## ETL → эмбеддинги → FAISS‑индекс

In [ ]:

import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

def load_catalog(xlsx_path):
    har = pd.read_excel(xlsx_path, sheet_name="характеристики")
    prices = pd.read_excel(xlsx_path, sheet_name="цены_скидки")
    df = har.merge(prices, on=["Бренд","Модель","Комплектация"], how="left")
    docs, meta = [], []
    for _, r in df.iterrows():
        parts = [
            f"{r['Бренд']} {r['Модель']} — {r['Комплектация']}",
            f"кузов: {r.get('Кузов','')}",
            f"привод: {r.get('Привод','')}",
            f"КПП: {r.get('Тип трансмиссии','')} ({r.get('Детали трансмиссии','')})",
            f"мощность: {r.get('Мощность, л.с.','')}",
            f"РРЦ: {r.get('Цена, руб','')}",
            f"скидки: TI {r.get('Скидка trade-in, руб','')}, CR {r.get('Скидка кредит, руб','')}, иной {r.get('Скидка иная, руб','')}"
        ]
        doc = ". ".join(str(x) for x in parts if pd.notna(x) and str(x)!='')
        docs.append(doc)
        meta.append({
            "brand": r["Бренд"], "model": r["Модель"], "trim": r["Комплектация"],
            "drive": r.get("Привод"), "gearbox": r.get("Тип трансмиссии"),
            "body": r.get("Кузов"), "power": r.get("Мощность, л.с."),
            "price": r.get("Цена, руб"), "ti": r.get("Скидка trade-in, руб"),
            "cr": r.get("Скидка кредит, руб"), "oth": r.get("Скидка иная, руб"),
            "total": r.get("Цена итого, руб"),
        })
    return df, docs, meta

catalog_df, docs, meta = load_catalog(XLSX_PATH)
print(f"Строк каталога: {len(catalog_df)}")

model = SentenceTransformer(EMBED_MODEL)
embs = model.encode(docs, batch_size=64, show_progress_bar=True, normalize_embeddings=True)
d = embs.shape[1]

index = faiss.IndexFlatIP(d)
index.add(embs)
print("FAISS готов:", index.ntotal, "векторов")


Строк каталога: 64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

FAISS готов: 64 векторов


## Поиск/фильтрация по каталогу

In [ ]:

# === Поиск/фильтрация по каталогу ===
import pandas as pd
import numpy as np
import re

def prefilter_mask(df, brand=None, max_price=None, drive=None, gearbox=None, body=None):
    """Жёсткие фильтры по колонкам перед векторным поиском."""
    mask = pd.Series([True] * len(df))
    if brand:
        mask &= df["Бренд"].fillna("").str.contains(brand, case=False, na=False)
    if max_price is not None:
        # если цена пуста — считаем её «очень большой», чтобы не пройти фильтр по max_price
        mask &= (df["Цена, руб"].fillna(10**12) <= max_price)
    if drive:
        mask &= df["Привод"].fillna("").str.contains(drive, case=False, na=False)
    if gearbox:
        mask &= df["Тип трансмиссии"].fillna("").str.contains(gearbox, case=False, na=False)
    if body:
        mask &= df["Кузов"].fillna("").str.contains(body, case=False, na=False)
    return mask

def retrieve_cars(query, top_k=20, **filters):
    """
    Векторный поиск среди предварительно отфильтрованных записей.
    Возвращает список кортежей: (similarity, row_index, meta_dict)
    """
    mask = prefilter_mask(catalog_df, **filters)
    idxs = np.where(mask.values)[0]
    if len(idxs) == 0:
        return []

    q_emb = model.encode([query], normalize_embeddings=True)  # SentenceTransformer
    sub = embs[idxs]  # embs — матрица эмбеддингов каталога (n x d)
    sims = (sub @ q_emb.T).ravel()  # косинус (т.к. нормализовано)

    ords = np.argsort(-sims)[:top_k]
    hits = []
    for pos in ords:
        i = idxs[pos]
        hits.append((float(sims[pos]), i, meta[i]))  # meta — сопутствующие поля на строку
    return hits

def safe_int(x, default=None):
    """Безопасно приводит к int. Понимает None/NaN/строки с пробелами и неразрывными пробелами."""
    try:
        if x is None:
            return default
        if isinstance(x, float) and np.isnan(x):
            return default
        x_str = str(x).replace(" ", "").replace("\xa0", "")
        return int(round(float(x_str)))
    except Exception:
        return default

def top3_human(hits):
    """
    Преобразует результаты retrieve_cars в удобный вид.
    Считает 'итого' только если есть РРЦ (price), скидки пустые считаются 0.
    """
    out, used = [], set()
    for sim, i, m in hits:
        key = (m.get("brand"), m.get("model"), m.get("trim"))
        if key in used:
            continue
        used.add(key)

        price_i = safe_int(m.get("price"), default=None)  # РРЦ может быть пустой
        ti_i    = safe_int(m.get("ti"),    default=0)
        cr_i    = safe_int(m.get("cr"),    default=0)
        oth_i   = safe_int(m.get("oth"),   default=0)
        itog    = price_i - ti_i - cr_i - oth_i if price_i is not None else None

        out.append({
            "similarity": round(sim, 3),
            "brand":   m.get("brand"),
            "model":   m.get("model"),
            "trim":    m.get("trim"),
            "drive":   m.get("drive"),
            "gearbox": m.get("gearbox"),
            "power":   m.get("power"),
            "price":   price_i,  # int или None
            "discounts": {"tradein": ti_i, "credit": cr_i, "other": oth_i},
            "total":   itog
        })
        if len(out) == 3:
            break
    return out

# Пример использования:
# hits = retrieve_cars(
#     "кроссовер до 3 млн с автоматом и полным приводом",
#     brand=None, max_price=3_000_000, drive="AWD|4WD", gearbox="AT|DCT", body="кроссовер"
# )
# top3 = top3_human(hits)
# top3


## RAG с использованием большой языковой модели (LLM)

**⚠️ ВАЖНО:** Перед использованием функций диалога убедитесь, что эта ячейка выполнена!
Функции LLM должны быть загружены до вызова `run_dialog()`.


In [ ]:
# === RAG с использованием LLM для генерации рекомендаций, отчетов и ссылок ===
import json
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from io import BytesIO
import base64

def format_car_data_for_llm(offers):
    """Форматирует данные об автомобилях для передачи в LLM."""
    formatted = []
    for i, car in enumerate(offers, 1):
        price_str = f"{car['price']:,}".replace(",", " ") if car['price'] else "не указана"
        total_str = f"{car['total']:,}".replace(",", " ") if car['total'] else "не рассчитана"
        ti_str = f"{car['discounts']['tradein']:,}".replace(",", " ") if car['discounts']['tradein'] else "0"
        cr_str = f"{car['discounts']['credit']:,}".replace(",", " ") if car['discounts']['credit'] else "0"
        oth_str = f"{car['discounts']['other']:,}".replace(",", " ") if car['discounts']['other'] else "0"

        car_info = f"""Автомобиль {i}:
- Бренд и модель: {car['brand']} {car['model']}
- Комплектация: {car['trim']}
- Привод: {car['drive']}
- Коробка передач: {car['gearbox']}
- Мощность: {car['power']} л.с.
- РРЦ: {price_str} ₽
- Скидки: Trade-in {ti_str} ₽, Кредит {cr_str} ₽, Прочие {oth_str} ₽
- Итоговая цена: {total_str} ₽
"""
        formatted.append(car_info)
    return "\n".join(formatted)

def generate_recommendations_with_llm(user_query: str, offers: list) -> str:
    """
    Генерирует персонализированные рекомендации с использованием LLM на основе данных из RAG.
    Это ключевая функция RAG - LLM получает релевантную информацию из базы знаний и генерирует ответ.
    """
    if not offers:
        return "К сожалению, по вашему запросу в каталоге не найдено подходящих автомобилей. Готов передать ваш запрос менеджеру для уточнения."

    car_data = format_car_data_for_llm(offers)

    system_prompt = """Ты - профессиональный консультант автосалона «АвтоЛидер».
Твоя задача - помочь клиенту выбрать автомобиль на основе данных из каталога.
ВАЖНО: Используй ТОЛЬКО информацию из предоставленных данных. Не придумывай характеристики, цены или модели, которых нет в данных.
Будь дружелюбным, профессиональным и конкретным. Выдели преимущества каждого автомобиля и помоги клиенту сделать выбор."""

    user_prompt = f"""Запрос клиента: "{user_query}"

Данные из каталога (найдено {len(offers)} вариантов):
{car_data}

Сгенерируй персонализированные рекомендации для клиента на основе этих данных.
Включи:
1. Краткое резюме запроса клиента
2. Описание каждого найденного автомобиля с акцентом на его преимущества
3. Сравнение вариантов (если их несколько)
4. Рекомендацию по выбору с обоснованием
5. Информацию о ценах и скидках

Ответ должен быть структурированным, понятным и убедительным. Используй только факты из предоставленных данных."""

    try:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.7,
            max_tokens=1500
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Ошибка при генерации рекомендаций через LLM: {e}")
        # Fallback на простое форматирование
        lines = []
        for i, o in enumerate(offers, 1):
            p = f"{o['price']:,}".replace(",", " ") if o['price'] else "не указана"
            tot = f" → итого {o['total']:,}".replace(",", " ") if o['total'] else ""
            spec = f"{o['gearbox']} / {o['drive']} / {o['power']} л.с."
            lines.append(f"{i}) {o['brand']} {o['model']} — {o['trim']} — {spec} — РРЦ {p} ₽{tot}")
        return "Топ‑3 варианта под ваш запрос:\n" + "\n".join(lines)

print("✅ Функции RAG с LLM загружены.")


✅ Функции RAG с LLM загружены.


## Время работы и SLA‑формулировки

In [ ]:

def is_open(now=None):
    now = now or datetime.now(TZ)
    t = now.time()
    return (t >= WORK_START) and (t <= WORK_END)

def is_late_cutoff(now=None):
    now = now or datetime.now(TZ)
    t = now.time()
    return (t >= LATE_CUTOFF)

def sla_phrase():
    if is_open():
        if is_late_cutoff():
            return ("в течение первых 10 минут работы автосалона. "
                    "Автосалон «АвтоЛидер» работает ежедневно с 9:00 до 21:00.")
        else:
            return "в течение 10 минут"
    else:
        return ("в течение первых 10 минут работы автосалона. "
                "Автосалон «АвтоЛидер» работает ежедневно с 9:00 до 21:00.")

print("Сейчас открыто:", is_open(), "| поздний порог:", is_late_cutoff())
print("SLA:", sla_phrase())


Сейчас открыто: False | поздний порог: True
SLA: в течение первых 10 минут работы автосалона. Автосалон «АвтоЛидер» работает ежедневно с 9:00 до 21:00.


## LangGraph: маршрутизация интентов и сценарии

In [ ]:

from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional, Dict, Any

class BotState(TypedDict):
    intent: Optional[str]
    brand: Optional[str]
    slots: Dict[str, Any]
    tries: int
    contact: Dict[str, Optional[str]]
    offers: list
    messages: list

def new_state():
    return BotState(
        intent=None, brand=None, slots={}, tries=0,
        contact={"name": None, "phone": None},
        offers=[], messages=[]
    )

def classify_intent(text: str) -> str:
    t = text.lower()
    if any(k in t for k in ["нов", "подобрать", "купить", "есть в наличии"]): return "NewCar"
    if any(k in t for k in ["с пробегом", "бу", "подержан"]): return "UsedCar"
    if any(k in t for k in ["запчаст", "детал", "поменять", "каталог запчастей"]): return "Parts"
    if any(k in t for k in ["крас", "вмятин", "кузов", "страховой случай"]): return "Body"
    if any(k in t for k in ["ремонт", "диагност", "то", "стук", "ошибк"]): return "Service"
    if any(k in t for k in ["бух", "закрывающ", "счет", "акт"]): return "Buch"
    if any(k in t for k in ["директор", "маркетинг", "кредитный отдел"]): return "VIP"
    return "Other"

def ingress(state: BotState, text: str) -> BotState:
    """Добавляет приветствие только если его еще нет в сообщениях."""
    if not state.get("messages") or not any("Здравствуйте" in msg for msg in state["messages"]):
        state["messages"].append('Здравствуйте, автосалон «АвтоЛидер»! Чем я могу вам помочь?')
    return state

def nlu_node(state: BotState, text: str) -> BotState:
    intent = classify_intent(text)
    state["intent"] = intent
    if "chery" in text.lower(): state["brand"]="CHERY"
    if "jetour" in text.lower(): state["brand"]="JETOUR"
    if "haval" in text.lower(): state["brand"]="HAVAL"
    return state

def newcar_flow(state: BotState, text: str) -> BotState:
    """
    Обработка запроса на подбор нового автомобиля с использованием RAG и LLM.
    LLM генерирует рекомендации на основе данных из базы знаний (каталога).
    """
    drive = "AWD|4WD" if "полный" in text.lower() else None
    gearbox = "AT|DCT" if any(x in text.lower() for x in ["автомат","робот","dct","at"]) else None
    budget = None
    import re
    m = re.search(r"(\d[\d\s]{2,})\s*р", text.lower())
    if m:
        try: budget = int(m.group(1).replace(" ",""))
        except: pass

    # RAG: поиск релевантных автомобилей в базе знаний
    hits = retrieve_cars(
        "подобрать автомобиль под запрос клиента",
        brand=state.get("brand"), max_price=budget, drive=drive, gearbox=gearbox, body="кроссовер|седан|пикап"
    )
    offers = top3_human(hits)
    state["offers"] = offers

    if offers:
        # RAG: использование LLM для генерации рекомендаций на основе данных из базы знаний
        try:
            # Проверяем, что функции LLM доступны
            if 'generate_recommendations_with_llm' not in globals():
                raise NameError("Функция generate_recommendations_with_llm не найдена. Убедитесь, что ячейка с функциями LLM выполнена.")

            print("🔄 Генерация рекомендаций через LLM...")
            recommendations = generate_recommendations_with_llm(text, offers)
            state["messages"].append(recommendations)
        except Exception as e:
            print(f"⚠️ Ошибка при использовании LLM: {e}")
            print("Используется fallback форматирование...")
            # Fallback на простое форматирование, если LLM недоступна
            lines = []
            for i, o in enumerate(offers, 1):
                p = f"{o['price']:,}".replace(",", " ") if o['price'] else "не указана"
                tot = f" → итого {o['total']:,}".replace(",", " ") if o['total'] else ""
                spec = f"{o['gearbox']} / {o['drive']} / {o['power']} л.с."
                lines.append(f"{i}) {o['brand']} {o['model']} — {o['trim']} — {spec} — РРЦ {p} ₽{tot}")
            state["messages"].append("Топ‑3 варианта под ваш запрос:\n" + "\n".join(lines))

        state["messages"].append("\nПредлагаю созвон с менеджером отдела продаж — удобно сейчас? "
                                 "Отправьте номер кнопкой «Поделиться контактом» или напишите его.")
    else:
        state["messages"].append("По вашему запросу в каталоге сейчас ничего не нашлось. "
                                 "Готов передать контакт менеджеру для уточнения.")
    return state

def usedcar_flow(state: BotState, text: str) -> BotState:
    state["messages"].append("Передаю ваш запрос в отдел автомобилей с пробегом (Neurobot_probeg). "
                             f"Менеджер свяжется {sla_phrase()}.")
    return state

def service_flow(state: BotState, text: str) -> BotState:
    state["messages"].append("Запрос по ремонту/ТО принят. "
                             "Уточните марку и VIN (если есть), передаю на СТО. "
                             f"Менеджер свяжется {sla_phrase()}.")
    return state

def body_flow(state: BotState, text: str) -> BotState:
    state["messages"].append("Запрос по кузовному ремонту принят (Neurobot_Kuzov_Chery). "
                             f"Менеджер свяжется {sla_phrase()}.")
    return state

def parts_flow(state: BotState, text: str) -> BotState:
    state["messages"].append("Запрос по запчастям принят. Напишите марку/модель/VIN и нужную деталь. "
                             f"Менеджер свяжется {sla_phrase()}.")
    return state

def buch_flow(state: BotState, text: str) -> BotState:
    state["messages"].append("Запрос передан в бухгалтерию (Neurobot_Buch). "
                             f"Специалист свяжется {sla_phrase()}.")
    return state

def vip_flow(state: BotState, text: str) -> BotState:
    state["messages"].append("Благодарю за обращение. Консультант свяжется с вами "
                             f"{sla_phrase()}.")
    return state

def other_flow(state: BotState, text: str) -> BotState:
    state["tries"] += 1
    if state["tries"] >= 2:
        state["messages"].append("Извините, я не могу вас понять, "
                                 f"{sla_phrase()}.")
    else:
        state["messages"].append("Уточните, пожалуйста: вы хотите подобрать новый автомобиль, "
                                 "оформить сервис/кузовной ремонт, купить запчасти или интересует авто с пробегом?")
    return state

def run_dialog(user_text: str, is_first_message: bool = True):
    """
    Запускает диалог с пользователем.
    is_first_message: True для первого сообщения (добавит приветствие), False для последующих.
    """
    st = new_state()
    if is_first_message:
        st = ingress(st, "")
    st = nlu_node(st, user_text)
    st = newcar_flow(st, user_text) if st["intent"]=="NewCar" else          usedcar_flow(st, user_text) if st["intent"]=="UsedCar" else          service_flow(st, user_text) if st["intent"]=="Service" else          body_flow(st, user_text) if st["intent"]=="Body" else          parts_flow(st, user_text) if st["intent"]=="Parts" else          buch_flow(st, user_text) if st["intent"]=="Buch" else          vip_flow(st, user_text) if st["intent"]=="VIP" else          other_flow(st, user_text)
    print("— Пользователь:", user_text)
    for m in st["messages"]:
        print("🤖", m)
    if st.get("offers"):
        print("\nПредложения (JSON):")
        import json; print(json.dumps(st["offers"], ensure_ascii=False, indent=2))

print("LangGraph каркас загружен.")


LangGraph каркас загружен.


## Демонстрация диалогов

In [ ]:

print("### Сценарий 1 — Новый авто:")
run_dialog("Хочу подобрать новый кроссовер до 3 000 000 р с автоматом и полным приводом.", is_first_message=True)

print("\n### Сценарий 2 — Сервис:")
run_dialog("Запишите на диагностику, ошибка на панели.", is_first_message=True)

print("\n### Сценарий 3 — Непонятный запрос:")
run_dialog("??", is_first_message=True)
run_dialog("что по поводу этого", is_first_message=False)  # Второе сообщение без приветствия


### Сценарий 1 — Новый авто:
🔄 Генерация рекомендаций через LLM...
— Пользователь: Хочу подобрать новый кроссовер до 3 000 000 р с автоматом и полным приводом.
🤖 Здравствуйте, автосалон «АвтоЛидер»! Чем я могу вам помочь?
🤖 Здравствуйте! Я рад, что вы обратились к нам за помощью в выборе нового кроссовера. Вы ищете автомобиль с автоматической коробкой передач и полным приводом в пределах 3 000 000 рублей. Я нашел несколько интересных вариантов для вас.

### 1. Краткое резюме запроса клиента
- **Бюджет:** до 3 000 000 ₽
- **Тип автомобиля:** кроссовер
- **Коробка передач:** автомат
- **Привод:** полный

### 2. Описание найденных автомобилей

**Автомобиль 1: CHERY TIGGO 7L**
- **Комплектация:** Ultra 1.6T DCT 2WD
- **Привод:** Передний (FWD)
- **Коробка передач:** DCT
- **Мощность:** 150 л.с.
- **Итоговая цена:** 2 870 000 ₽
- **Преимущества:** Стильный дизайн и достаточно мощный двигатель в 150 л.с. Прекрасно подойдет для городской эксплуатации. 

**Автомобиль 2: CHERY TIGGO 4**
- **Ком